<style>
    h1 {
        color: white;
        text-variant: italic;
        text-decoration: 2px underline;
    }
</style>
<h1>Finding an image window from main image</h1>

In [1]:
import numpy as np;

import skimage;
import skimage.io;
import skimage.transform;
import skimage.util;
import skimage.draw;

from SpiralGen import get_spiral;

### Loading a sample set of images

In [2]:
img_4x = skimage.util.img_as_float(skimage.io.imread(r"T 26A-22 ductal CA/20220402091557029.tif"));
img_10x = skimage.util.img_as_float(skimage.io.imread(r"T 26A-22 ductal CA/20220402091730689.tif"));
img_40x = skimage.util.img_as_float(skimage.io.imread(r"T 26A-22 ductal CA/20220402091841778.tif"));

### Following functions returns pivot(center) of the window

In [3]:
def find_location(
        main_image: np.ndarray,
        window_image: np.ndarray,
    ) -> tuple:
    main_image_height: int = main_image.shape[0];
    main_image_width: int = main_image.shape[1];

    window_image_height: int = window_image.shape[0];
    window_image_width: int = window_image.shape[1];
    semi_window_image_height: int = window_image_height // 2;
    semi_window_image_width: int = window_image_width // 2;

    start_x: int;
    start_y: int;
    ratio: int = 2; # Use power of 2, i.e. 2,4,8,16....
    scanthreshold: int = 64

    if window_image_width > scanthreshold and window_image_height > scanthreshold:
        start_x, start_y = find_location(
            skimage.transform.rescale(main_image, 1/ratio, channel_axis=2),
            skimage.transform.rescale(window_image, 1/ratio, channel_axis=2),
        );
        start_x *= ratio;
        start_y *= ratio;
        flag: bool = True;
    else:
        start_x, start_y = main_image_height//2, main_image_width//2;
        flag: bool = False;

    min_avg_diff: float = 999999;
    min_diff_at: tuple = (0,0);
    img_slice: np.ndarray;
    
    r = ( (ratio*2+2) if flag else min(main_image_height - window_image_height, main_image_width - window_image_width));

    for x, y in get_spiral(start_x, start_y, step_end=r):
        img_slice = main_image[
            x - semi_window_image_height: x + semi_window_image_height + (1 if window_image_height%2 else 0),
            y - semi_window_image_width: y + semi_window_image_width + (1 if window_image_width%2 else 0)
        ];
        
        current_diff_img: np.ndarray = np.abs(img_slice - window_image);
        curr_min_avg_diff: float = np.sum(current_diff_img);

        if curr_min_avg_diff < min_avg_diff:
            min_avg_diff = curr_min_avg_diff;
            min_diff_at = x, y;
            skimage.io.imsave(f"tests//{ratio}_{min_diff_at}_{min_avg_diff}.bmp", skimage.util.img_as_ubyte(current_diff_img));


    return min_diff_at;

In [4]:
x1, y1 = find_location(img_4x, skimage.transform.rescale(img_10x, 1/2.5, channel_axis=2))
x2, y2 = find_location(img_10x, skimage.transform.rescale(img_40x, 1/4, channel_axis=2))